In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import matplotlib.pyplot as plt # for plotting
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns


from datetime import timedelta
from datetime import date
from datetime import datetime
 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


import os
#print(os.listdir("../input"))
plt.rcParams['figure.figsize'] = [16, 10]
%matplotlib inline

# Any results you write to the current directory are saved as output.

____
# **1.DATA LOADING**

In [ ]:
dftrain = pd.read_csv('../input/train.csv')
dftest = pd.read_csv('../input/test.csv')
#dftrain = pd.read_csv('./train.csv')
#dftest = pd.read_csv('./test.csv')
dftrain.head()

In [ ]:
dftest.head()

> **As we can see, test.csv do not have trip_duration and dropoff_datetime.**

______

In [ ]:
dftrain.corr()

In [ ]:
dftrain.shape

_____

In [ ]:
dftrain.dtypes

____

In [ ]:
dftrain.any().isnull()


> **There are no missing information in this dataset.**
____

## **2 - Stat overview**

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
dftrain.describe()


### **Curious values: **
The `trip_duration` `min` is 1 second ?  Can a  taxi make a trip of 1 second ? 
Trip  `max` of  3526282s ? Passenger_count min of 0 ?


> **These values seem to be outliers. Lets focus on them.**


____

## **3.Convert Date Object to DateTime**

In [ ]:
dftrain['pickup_datetime'] = pd.to_datetime(dftrain['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
dftrain['dropoff_datetime'] = pd.to_datetime(dftrain['dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')

dftest['pickup_datetime'] = pd.to_datetime(dftest['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

dftrain.dtypes, dftest.dtypes

_____

##  **4.Trip_duration Outliers**

In [ ]:
plt.subplots(figsize=(18,10))
plt.title("Détection des outliers")
dftrain.boxplot();

>  There is somme `trip duration` outliers


In [ ]:
#dftrain = dftrain[dftrain['trip_duration'] > 300 ] 
dftrain = dftrain[dftrain['trip_duration'] < 10000 ] 

> **I removed all data which `trip_duration` is less  than 5 minutes and all greater than 12h day duration. Because a taxi can be reserved multi hours**


In [ ]:
# Creating new columns for minutes, hours & days
#dftrain['trip_duration_min'] = dftrain['trip_duration']
#dftrain['trip_duration_hour'] = dftrain['trip_duration']/3600
#dftrain['trip_duration_days'] = dftrain['trip_duration']/(3600*24)

In [ ]:
dftrain.shape

In [ ]:
plt.figure(figsize=(16,8))

plt.hist(dftrain['trip_duration'], bins=50)

plt.xlabel('trip duration')
plt.ylabel('number of train records')
plt.title('Histogram of trip duration repartition')
#plt.text(100, .025, r'$\mu=959,\ \sigma=5237.4$')
#plt.axis([-500, 10000, 0, 150000])
plt.grid(True)
plt.show();

> As we can see, most of trips have a duration less thans `2000 seconds`, `33 mins`. 

In [ ]:
plt.subplots(figsize=(16,6))
dftrain['trip_duration_log'] = np.log(dftrain['trip_duration'].values)
plt.hist(dftrain['trip_duration_log'], bins=50)
plt.xlabel('trip duration')
plt.ylabel('number of train records')
plt.title('Histogram of trip duration log repartition')
plt.show();

____

## **5.Passengers cleaning**

In [ ]:
#dftrain = dftrain[dftrain['passenger_count']<= 5 ]
#dftrain = dftrain[dftrain['passenger_count']>= 1 ]

A Taxi can only take 4 persons that why we remove all others.

In [ ]:
dftrain.shape

In [ ]:
f, ax = plt.subplots(figsize=(10.5, 10.5))
sns.despine(f, left=True, bottom=True)
sns.scatterplot(x="passenger_count", y="trip_duration",
                data=dftrain);
plt.title('Repartition time duration by number of passenger ');

In [ ]:
f, ax = plt.subplots(figsize=(10.5, 10.5))
sns.set(style="darkgrid")
sns.lineplot(x="passenger_count", y="trip_duration",data=dftrain);

> **I removed all data which *Passengers_count* is  greater than 6.** 
___

## **6.Geo Cleaning**

In [ ]:
dftrain = dftrain[dftrain['pickup_longitude'] >= -74.03]
dftrain = dftrain[dftrain['pickup_longitude'] <= -73.75]
dftrain = dftrain[dftrain['pickup_latitude'] <= 40.85]
dftrain = dftrain[dftrain['pickup_latitude'] >= 40.63]
dftrain = dftrain[dftrain['dropoff_longitude'] <= -73.75]
dftrain = dftrain[dftrain['dropoff_longitude'] >= -74.03]
dftrain = dftrain[dftrain['dropoff_latitude'] <= 40.85]
dftrain = dftrain[dftrain['dropoff_latitude'] >= 40.63]

In [ ]:
dftrain.shape

  > **For beeing more coherent to Taxi trips in New York I limited the geographic coordonates to the city**

In [ ]:
fig, ax = plt.subplots(figsize=(15,12))
ax.scatter(dftrain['pickup_longitude'],dftrain['pickup_latitude'], s=0.0002, alpha=1);
plt.ylim(40.6, 40.86)
plt.xlim(-74.05,-73.75)
plt.title('Concentration of trips in NY')

____

## **7. Month durations**

In [ ]:
dftrain['month'] = dftrain['pickup_datetime'].dt.month
dftrain['week'] = dftrain['pickup_datetime'].dt.week
dftrain['weekday'] = dftrain['pickup_datetime'].dt.weekday
dftrain['hours'] = dftrain['pickup_datetime'].dt.hour
#dftrain['minutes'] = dftrain['pickup_datetime'].dt.minute
#dftrain['day'] = dftrain['pickup_datetime'].dt.day

In [ ]:
dftest['month'] = dftest['pickup_datetime'].dt.month
dftest['week'] = dftest['pickup_datetime'].dt.week
dftest['weekday'] = dftest['pickup_datetime'].dt.weekday
dftest['hours'] = dftest['pickup_datetime'].dt.hour
#dftest['day'] = dftest['pickup_datetime'].dt.day

In [ ]:
dftrain.groupby(['month'])['month'].count().plot(kind='bar', figsize=(15, 5));
plt.title('Number of value per month ')
plt.xlabel('trip_duration')
plt.ylabel('number of trips records')
plt.title('Repartition of trips each month')
plt.grid(True)
plt.show();

In [ ]:
dftrain.groupby(['month'])['trip_duration'].mean().plot(kind='bar', figsize=(15, 5));
plt.title(' Mean duration per month');

________

In [ ]:
dftrain.groupby(['weekday'])['trip_duration'].count().plot(kind='bar', figsize=(15, 5));
plt.ylabel('number of trips');
plt.title('Number of trip each days of week')

In [ ]:
dftrain.groupby(['weekday'])['trip_duration'].mean().plot(kind='bar', figsize=(15, 5));
plt.ylabel('number of trips');
plt.title('Number of trip each days of week')

____

In [ ]:
dftrain.groupby(['week'])['trip_duration'].count().plot(kind='bar', figsize=(26, 5));
plt.ylabel('number of trips');
plt.title('Number of trips each week');

In [ ]:
dftrain.groupby(['week'])['trip_duration'].mean().plot(kind='bar', figsize=(26, 5));
plt.ylabel('number of trips');
plt.title('Mean of trips each week');

______

In [ ]:
dftrain.groupby(['hours'])['trip_duration'].count().plot(kind='bar', figsize=(15, 5));
plt.title('Number of trip per Hours ')
plt.xlabel(' Hours')
plt.ylabel('number of trips');

In [ ]:
dftrain.groupby(['hours'])['trip_duration'].mean().plot(kind='bar', figsize=(15, 5));
plt.title('Mean of trip per Hours ')
plt.xlabel('Hours')
plt.ylabel('number of trips')
plt.grid(True)
plt.show();

_____
## **Distance**

In [ ]:
dftrain['dist'] = np.sqrt((dftrain['pickup_latitude']-dftrain['dropoff_latitude'])**2
                        + (dftrain['pickup_longitude']-dftrain['dropoff_longitude'])**2)
dftest['dist'] = np.sqrt((dftest['pickup_latitude']-dftest['dropoff_latitude'])**2
                        + (dftest['pickup_longitude']-dftest['dropoff_longitude'])**2)


___

In [ ]:
dftrain['store_and_fwd_flag'] = dftrain['store_and_fwd_flag'].astype('category').cat.codes
dftest['store_and_fwd_flag'] = dftest['store_and_fwd_flag'].astype('category').cat.codes
dftrain.head()

In [ ]:
dftrain['store_and_fwd_flag'].value_counts()

## **8. Model **

In [ ]:
dftrain.head()

In [ ]:
dftrain['pickup_datetime'] = dftrain['pickup_datetime'].astype(int)
dftrain.head()

In [ ]:
dftest['pickup_datetime'] = dftest['pickup_datetime'].astype(int)
dftest.head()

In [ ]:
selection = ["vendor_id", "passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude","month", "weekday", "hours", "dist", "store_and_fwd_flag"]

# 1 mettre en commun les variable : 
#select_train = ['pickup_datetime', "passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude"]
#select_test = ['pickup_datetime', "passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude"]

# préparation des variable d'entrainement
X = dftrain[selection]
# target d'entrainement : 
y = dftrain["trip_duration_log"]  # target

# Donnée de test à predir
X_test = dftest[selection]

In [ ]:
X_test.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=4, min_samples_split=15, n_jobs=-1)

In [ ]:
m1_scores = cross_val_score(rf, X, y, cv=5, scoring="neg_mean_squared_log_error", n_jobs=-1)

In [ ]:
for i in range(len(m1_scores)):
    m1_scores[i] = np.sqrt(abs(m1_scores[i]))
print(m1_scores)

In [ ]:
rf.fit(X, y)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
my_submission = pd.DataFrame({'id': dftest.id, 'trip_duration': np.exp(y_pred)})
my_submission.head()

In [ ]:
my_submission.to_csv('submission.csv',index=False)